**//IMPORTS**

In [1]:
import os, gzip, requests, time
import pandas as pd
import pathlib, sys

from datetime import datetime, timezone
from dotenv import load_dotenv         
from pathlib import Path
from opensubtitlescom import OpenSubtitles
from xmlrpc.client import ServerProxy, Error as XMLRPCError

project_root = pathlib.Path().resolve().parents[0] 
print(f"Project root: {project_root}")
sys.path.insert(0, str(project_root / "src"))
from extract.access_open_subtitles import (fetch_subtitles_for_br, save_batch, save_batch)
from catalog import load_intersection_from_catalog


Project root: /home/rofarate/Thesis
OpenSubtitles login failed: 429 Client Error: Too Many Requests for url: https://api.opensubtitles.com/api/v1/login → continuing without token.


**//CONFIGS**

In [ ]:
# load_dotenv()

# API_BASE = "https://api.opensubtitles.com/api/v1"
# API_KEY = os.getenv("OPENSUBTITLES_API_KEY")
# USERNAME = os.getenv("OPENSUBTITLES_USER", "")
# PASSWORD = os.getenv("OPENSUBTITLES_PASS", "")
# USER_AGENT = "MySubtitleApp/1.0"

# # REST headers
# # REST_HEADERS = {
# #     "Api-Key":     API_KEY,
# #     "User-Agent":  USER_AGENT,
# #     "Content-Type":"application/json"
# # }

# # HEAD = {
# #     "Api-Key": os.getenv("OPENSUBTITLES_API_KEY"),
# #     "User-Agent": "TeseCollector/0.1",
# #     "Accept": "application/json",
# # }

# BASE_HEADERS = {
#     "Api-Key":     API_KEY,
#     "User-Agent":  "MySubtitleApp/1.0",          # <- descriptive!
#     "Accept":      "application/json",           # <- important
#     "Content-Type":"application/json",
# }
# AUTH_HEADERS = dict(BASE_HEADERS)       # will gain 'Authorization' below

# # XML-RPC client (fallback path)
# ost = OpenSubtitles(user_agent=USER_AGENT, api_key=API_KEY)
# # optional login for higher quotas
# if USERNAME and PASSWORD:
#     try:
#         ost.login(USERNAME, PASSWORD)
#     except Exception:
#         pass

# # YEARS AND LANGUAGES

# YEARS = range(2023, 2024)          
# LANGS = {"pt-br", "pt-pt"}             
# by_lang = {lang: {} for lang in LANGS}

# TIMEOUT = 15



In [2]:
OUTPUT_DIR_BR = Path("../data/raw/test_br_subs")
OUTPUT_DIR_PT = Path("../data/raw/test_pt_subs")
os.makedirs(OUTPUT_DIR_BR, exist_ok=True)
os.makedirs(OUTPUT_DIR_PT, exist_ok=True)



**//MOVIES IN OPENSUBTITLES IN PT AND BR**

In [ ]:
# import requests, time, random

# MAX_RETRIES   = 5          # try the same request up to 5 times
# BACKOFF_BASE  = 1.5        # exponential factor   (1.5, 2, 3 … all fine)
# JITTER        = 0.3        # add ±30 % random jitter to avoid thundering herd

# def safe_get(url, headers=None, params=None, timeout=15):
#     """GET with automatic retries on 5xx / network errors."""
#     delay = 1.0                           # initial wait (s) before retry
#     for attempt in range(1, MAX_RETRIES + 1):
#         try:
#             r = requests.get(url, headers=headers, params=params, timeout=timeout)
#             # retry only on 5xx
#             if 500 <= r.status_code < 600:
#                 raise requests.HTTPError(f"{r.status_code} {r.reason}", response=r)
#             return r                      # success
#         except (requests.ConnectionError,
#                 requests.Timeout,
#                 requests.HTTPError) as e:
#             if attempt == MAX_RETRIES:
#                 raise                     # bubble up after last attempt
#             sleep_for = delay * (1 + JITTER * (random.random() * 2 - 1))
#             print(f"[retry {attempt}/{MAX_RETRIES}] {e} – sleeping {sleep_for:.2f}s")
#             time.sleep(sleep_for)
#             delay *= BACKOFF_BASE         # exponential back-off


In [ ]:
# for lang in LANGS:
#     for year in YEARS:
#         page = 1
#         while True:
#             q = {"languages": lang, "year": year, "type": "movie", "page": page}
#             r = safe_get(f"{API_BASE}/subtitles", headers=BASE_HEADERS, params=q)
#             payload = r.json()
#             data = payload["data"]
#             if not data:
#                 break
#             for sub in data:
#                 feat = sub["attributes"]["feature_details"]
#                 by_lang[lang][feat["imdb_id"]] = feat["title"]
#             page += 1
#             time.sleep(1)                           # stay under 40 req / 10 s
#             if page > payload["total_pages"]:
#                 break



In [ ]:
# MOVIES = pd.read_csv("tmdb_movies_2024_with_imdb.csv")
# MOVIES = MOVIES[MOVIES["imdb_id"].notna()]
# MOVIES = MOVIES.to_dict(orient="records")

# MOVIES = [
#     {"imdb_id": "tt14513804", "title": "Captain America: Brave New World"},
#     {"imdb_id": "tt34463310", "title": "Detective Chinatown 1900"},
# ]

# both_langs = set(by_lang["pt-br"]).intersection(by_lang["pt-pt"])
# movies_by_id = {imdb: by_lang["pt-pt"][imdb] for imdb in both_langs}

# movies_df = (
#     pd.DataFrame.from_dict(movies_by_id, orient="index", columns=["title"])
#       .reset_index()
#       .rename(columns={"index": "imdb_id"})
# )

# movies_df

,imdb_id,title
0,28524545,Mystery Island
1,26931203,Barbie: Skipper and the Big Babysitting Adventure
2,20113412,The Convert
3,24082438,20 Days in Mariupol
4,9658388,Praise This
...,...,...
663,15845360,Dance First
664,4225012,A Little White Lie
665,13871094,Stamped from the Beginning
666,22054902,Ivalu


**//MAIN CODE**

In [8]:
movies_df = load_intersection_from_catalog(year=2023, n=10)

# download first 10 BR files
save_batch(movies_df, lang="pt-BR", n=1)

# download first 10 PT-PT files
save_batch(movies_df, lang="pt-PT", n=1)


Download refused: {'requests': 5, 'remaining': 0, 'message': 'You have downloaded your allowed 5 subtitles for 24h.Your quota will be renewed in 02 hours and 00 minutes (2025-07-07 23:59:59 UTC) ts=1751925595 ', 'reset_time': '02 hours and 00 minutes', 'reset_time_utc': '2025-07-07T23:59:59.000Z', 'uk': 'app_125690_85.242.215.63', 'uid': 2, 'ts': 1751925595}
✗ No pt-BR subs for 14230458 (Poor Things)
Download refused: {'requests': 5, 'remaining': 0, 'message': 'You have downloaded your allowed 5 subtitles for 24h.Your quota will be renewed in 02 hours and 00 minutes (2025-07-07 23:59:59 UTC) ts=1751925598 ', 'reset_time': '02 hours and 00 minutes', 'reset_time_utc': '2025-07-07T23:59:59.000Z', 'uk': 'app_125690_85.242.215.63', 'uid': 2, 'ts': 1751925598}
✗ No pt-PT subs for 14230458 (Poor Things)
